In [5]:
import sys
sys.path.append(r"C:\Users\PMLS\timeseires")

In [6]:
from timeseires.utils import t_v_t_split as sp
import numpy as np
import pandas as pd
from pandas import read_csv
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
import sys

In [5]:
dt = pd.read_csv('cleaned_citylearn_data_all_outliers_handled.csv')

In [7]:
train_set , validation_set , test_set = sp.t_v_t(dt,70,20,10)

In [22]:
train_set.shape

(6132, 29)

In [20]:
train_set

,non_shiftable_load,outdoor_dry_bulb_temperature_predicted_1,outdoor_dry_bulb_temperature_predicted_2,outdoor_dry_bulb_temperature_predicted_3,outdoor_relative_humidity_predicted_1,outdoor_relative_humidity_predicted_2,outdoor_relative_humidity_predicted_3,diffuse_solar_irradiance_predicted_1,diffuse_solar_irradiance_predicted_2,diffuse_solar_irradiance_predicted_3,...,carbon_intensity,outdoor_relative_humidity,diffuse_solar_irradiance,direct_solar_irradiance,month_sin,month_cos,hour_sin,hour_cos,day_type_sin,day_type_cos
0,2.188750,18.3,22.8,20.0,81,68,81,25,964,0,...,0.170724,84,0,0,-0.500000,-0.866025,-2.449294e-16,1.000000,-2.449294e-16,1.000000
1,1.370667,19.4,22.8,19.4,79,71,87,201,966,0,...,0.157319,79,0,0,-0.866025,-0.500000,2.588190e-01,0.965926,7.818315e-01,0.623490
2,1.101250,21.1,22.2,19.4,73,73,87,420,683,0,...,0.154503,78,0,0,-0.866025,-0.500000,5.000000e-01,0.866025,7.818315e-01,0.623490
3,0.543217,22.2,22.8,19.4,71,71,90,554,522,0,...,0.150945,78,0,0,-0.866025,-0.500000,7.071068e-01,0.707107,7.818315e-01,0.623490
4,0.542833,21.7,22.2,18.9,73,71,90,778,444,0,...,0.151820,78,0,0,-0.866025,-0.500000,8.660254e-01,0.500000,7.818315e-01,0.623490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6127,0.770867,17.8,14.4,13.9,65,84,80,937,9,82,...,0.144597,75,37,1,0.866025,-0.500000,9.659258e-01,-0.258819,-4.338837e-01,-0.900969
6128,0.649450,17.8,14.4,14.4,65,84,78,862,0,164,...,0.151096,72,210,8,0.866025,-0.500000,8.660254e-01,-0.500000,-4.338837e-01,-0.900969
6129,0.421517,17.2,13.9,16.7,65,83,70,724,0,243,...,0.147093,70,212,0,0.866025,-0.500000,7.071068e-01,-0.707107,-4.338837e-01,-0.900969
6130,0.412917,17.2,13.3,17.2,63,87,70,529,0,662,...,0.144542,65,653,470,0.866025,-0.500000,5.000000e-01,-0.866025,-4.338837e-01,-0.900969


In [11]:
train_set.shape,validation_set.shape,test_set.shape

((6132, 26), (1752, 26), (876, 26))

In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Load dataset
dt = pd.read_csv('cleaned_citylearn_data_all_outliers_handled.csv')

# Check if datetime column exists (replace 'datetime' with your column name)
if 'datetime' in dt.columns:
    dt['datetime'] = pd.to_datetime(dt['datetime'])
    dt['month'] = dt['datetime'].dt.month
    dt['hour'] = dt['datetime'].dt.hour
    dt['day_type'] = dt['datetime'].dt.dayofweek  # Monday=0, Sunday=6
else:
    print("No datetime column found; please ensure temporal features exist")

# Now split dataset into train, validation, test
train_set, validation_set, test_set = sp.t_v_t(dt, 70, 20, 10)

# Cyclical encoding function
def cyclical_encode(df, column, max_value):
    df[f"{column}_sin"] = np.sin(2 * np.pi * df[column] / max_value)
    df[f"{column}_cos"] = np.cos(2 * np.pi * df[column] / max_value)
    return df

# Apply cyclical encoding only if columns exist
for col, max_val in [('month', 12), ('hour', 24), ('day_type', 7)]:
    if col in test_set.columns:
        test_set = cyclical_encode(test_set, col, max_val)
    else:
        print(f"Warning: Column '{col}' not found in train_set")

# Drop original temporal columns if they exist
for col in ['month', 'hour', 'day_type']:
    if col in test_set.columns:
        test_set.drop(columns=[col], inplace=True)

# Standard scaling for weather features
weather_features = [
    'outdoor_dry_bulb_temperature', 'outdoor_dry_bulb_temperature_predicted_1','outdoor_dry_bulb_temperature_predicted_2','outdoor_dry_bulb_temperature_predicted_3',
    'outdoor_relative_humidity', 'outdoor_relative_humidity_predicted_1','outdoor_relative_humidity_predicted_2','outdoor_relative_humidity_predicted_3',
    'diffuse_solar_irradiance', 'diffuse_solar_irradiance_predicted_1','diffuse_solar_irradiance_predicted_2','diffuse_solar_irradiance_predicted_3',
    'direct_solar_irradiance', 'direct_solar_irradiance_predicted_1','direct_solar_irradiance_predicted_2','direct_solar_irradiance_predicted_3'
]

# Check columns before scaling
weather_features = [f for f in weather_features if f in test_set.columns]
weather_scaler = StandardScaler()
test_set[weather_features] = weather_scaler.fit_transform(test_set[weather_features])

# Min-Max scaling for economic/environmental features
econ_env_features = [
    'non_shiftable_load',
    'electricity_pricing', 'electricity_pricing_predicted_1','electricity_pricing_predicted_2','electricity_pricing_predicted_3',
    'solar_generation',
    'carbon_intensity'
]

econ_env_features = [f for f in econ_env_features if f in validation_set.columns]
econ_env_scaler = MinMaxScaler()
test_set[econ_env_features] = econ_env_scaler.fit_transform(test_set[econ_env_features])

# Save normalized train_set
train_set.to_csv(r'C:\Users\PMLS\shahid (ML-lab)\test_data.csv', index=False)
print("Normalized train_set saved successfully.")


No datetime column found; please ensure temporal features exist
Normalized train_set saved successfully.


C:\Users\PMLS\AppData\Local\Temp\ipykernel_3068\346147606.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{column}_sin"] = np.sin(2 * np.pi * df[column] / max_value)
C:\Users\PMLS\AppData\Local\Temp\ipykernel_3068\346147606.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{column}_cos"] = np.cos(2 * np.pi * df[column] / max_value)
C:\Users\PMLS\AppData\Local\Temp\ipykernel_3068\346147606.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [38]:
train_set

,non_shiftable_load,outdoor_dry_bulb_temperature_predicted_1,outdoor_dry_bulb_temperature_predicted_2,outdoor_dry_bulb_temperature_predicted_3,outdoor_relative_humidity_predicted_1,outdoor_relative_humidity_predicted_2,outdoor_relative_humidity_predicted_3,diffuse_solar_irradiance_predicted_1,diffuse_solar_irradiance_predicted_2,diffuse_solar_irradiance_predicted_3,...,carbon_intensity,outdoor_relative_humidity,diffuse_solar_irradiance,direct_solar_irradiance,month_sin,month_cos,hour_sin,hour_cos,day_type_sin,day_type_cos
0,0.751244,0.516779,1.703822,0.971357,0.496457,-0.205161,0.495388,-0.587570,3.044975,-0.682668,...,0.513946,0.657503,-0.682997,-0.651088,-0.500000,-0.866025,-2.449294e-16,1.000000,-2.449294e-16,1.000000
1,0.470454,0.806284,1.703822,0.813139,0.388581,-0.043375,0.818916,0.092300,3.052710,-0.682668,...,0.445284,0.387821,-0.682997,-0.651088,-0.866025,-0.500000,2.588190e-01,0.965926,7.818315e-01,0.623490
2,0.377982,1.253700,1.545821,0.813139,0.064951,0.064482,0.818916,0.938275,1.958238,-0.682668,...,0.430858,0.333884,-0.682997,-0.651088,-0.866025,-0.500000,5.000000e-01,0.866025,7.818315e-01,0.623490
3,0.186448,1.543204,1.703822,0.813139,-0.042925,-0.043375,0.980681,1.455903,1.335588,-0.682668,...,0.412636,0.333884,-0.682997,-0.651088,-0.866025,-0.500000,7.071068e-01,0.707107,7.818315e-01,0.623490
4,0.186317,1.411611,1.545821,0.681292,0.064951,-0.043375,0.980681,2.321192,1.033931,-0.682668,...,0.417117,0.333884,-0.682997,-0.651088,-0.866025,-0.500000,8.660254e-01,0.500000,7.818315e-01,0.623490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6127,0.264584,0.385186,-0.508191,-0.637187,-0.366555,0.657697,0.441466,2.935393,-0.648384,-0.365078,...,0.380122,0.172074,-0.539742,-0.647669,0.866025,-0.500000,9.659258e-01,-0.258819,-4.338837e-01,-0.900969
6128,0.222911,0.385186,-0.508191,-0.505340,-0.366555,0.657697,0.333623,2.645675,-0.683191,-0.047487,...,0.413408,0.010265,0.130070,-0.623740,0.866025,-0.500000,8.660254e-01,-0.500000,-4.338837e-01,-0.900969
6129,0.144677,0.227275,-0.639858,0.101161,-0.366555,0.603768,-0.097748,2.112596,-0.683191,0.258484,...,0.392909,-0.097608,0.137813,-0.651088,0.866025,-0.500000,7.071068e-01,-0.707107,-4.338837e-01,-0.900969
6130,0.141725,0.227275,-0.797859,0.233009,-0.474431,0.819482,-0.097748,1.359330,-0.683191,1.881294,...,0.379843,-0.367291,1.845253,0.955590,0.866025,-0.500000,5.000000e-01,-0.866025,-4.338837e-01,-0.900969


In [40]:
train_set.shape

(6132, 29)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Load dataset
dt = pd.read_csv('cleaned_citylearn_data_all_outliers_handled.csv')

# Extract datetime features
if 'datetime' in dt.columns:
    dt['datetime'] = pd.to_datetime(dt['datetime'])
    dt['month'] = dt['datetime'].dt.month
    dt['hour'] = dt['datetime'].dt.hour
    dt['day_type'] = dt['datetime'].dt.dayofweek  # Monday=0, Sunday=6
else:
    print("No datetime column found; please ensure temporal features exist")

# Split dataset into train, validation, test using your function sp.t_v_t()
train_set, validation_set, test_set = sp.t_v_t(dt, 70, 20, 10)

# ---- Cyclical Encoding Function ----
def cyclical_encode(df, column, max_value):
    df[f"{column}_sin"] = np.sin(2 * np.pi * df[column] / max_value)
    df[f"{column}_cos"] = np.cos(2 * np.pi * df[column] / max_value)
    return df

# Apply cyclical encoding to all sets for same columns
for dataset in [train_set, validation_set, test_set]:
    for col, max_val in [('month', 12), ('hour', 24), ('day_type', 7)]:
        if col in dataset.columns:
            cyclical_encode(dataset, col, max_val)
    # Drop original temporal columns after encoding
    dataset.drop(columns=['month', 'hour', 'day_type'], errors='ignore', inplace=True)

# Define feature groups
weather_features = [
    'outdoor_dry_bulb_temperature', 'outdoor_dry_bulb_temperature_predicted_1','outdoor_dry_bulb_temperature_predicted_2','outdoor_dry_bulb_temperature_predicted_3',
    'outdoor_relative_humidity', 'outdoor_relative_humidity_predicted_1','outdoor_relative_humidity_predicted_2','outdoor_relative_humidity_predicted_3',
    'diffuse_solar_irradiance', 'diffuse_solar_irradiance_predicted_1','diffuse_solar_irradiance_predicted_2','diffuse_solar_irradiance_predicted_3',
    'direct_solar_irradiance', 'direct_solar_irradiance_predicted_1','direct_solar_irradiance_predicted_2','direct_solar_irradiance_predicted_3'
]

econ_env_features = [
    'non_shiftable_load',
    'electricity_pricing', 'electricity_pricing_predicted_1','electricity_pricing_predicted_2','electricity_pricing_predicted_3',
    'solar_generation',
    'carbon_intensity'
]

# Filter features to those that actually exist in train set (to avoid errors)
weather_features = [f for f in weather_features if f in train_set.columns]
econ_env_features = [f for f in econ_env_features if f in train_set.columns]

# ---- Scaling ----

# 1) Weather features - StandardScaler (fit on train, transform on all)
weather_scaler = StandardScaler()
train_set[weather_features] = weather_scaler.fit_transform(train_set[weather_features])
validation_set[weather_features] = weather_scaler.transform(validation_set[weather_features])
test_set[weather_features] = weather_scaler.transform(test_set[weather_features])

# 2) Economic/Environmental features - MinMaxScaler (fit on train, transform on all)
econ_env_scaler = MinMaxScaler()
train_set[econ_env_features] = econ_env_scaler.fit_transform(train_set[econ_env_features])
validation_set[econ_env_features] = econ_env_scaler.transform(validation_set[econ_env_features])
test_set[econ_env_features] = econ_env_scaler.transform(test_set[econ_env_features])

# Save normalized datasets
train_set.to_csv(r'C:\Users\PMLS\shahid (ML-lab)\train_data.csv', index=False)
validation_set.to_csv(r'C:\Users\PMLS\shahid (ML-lab)\validation_data.csv', index=False)
test_set.to_csv(r'C:\Users\PMLS\shahid (ML-lab)\test_data.csv', index=False)

print("Normalized train, validation, and test datasets saved successfully.")


In [20]:
F = pd.read_csv('test_data.csv')

In [22]:
# Load your Excel file

#df = pd.read_excel('merged_citylearn_data.xlsx')
# Load dataset
#dt = pd.read_csv('train_data.csv')

# Remove columns E, F, G (index 4, 5, 6 in 0-based indexing)
# Method 1: By column names (recommended)
columns_to_remove = [
    'outdoor_dry_bulb_temperature_predicted_1','outdoor_dry_bulb_temperature_predicted_2','outdoor_dry_bulb_temperature_predicted_3',
    'outdoor_relative_humidity_predicted_1','outdoor_relative_humidity_predicted_2','outdoor_relative_humidity_predicted_3',
     'diffuse_solar_irradiance_predicted_1','diffuse_solar_irradiance_predicted_2','diffuse_solar_irradiance_predicted_3',
     'direct_solar_irradiance_predicted_1','direct_solar_irradiance_predicted_2','direct_solar_irradiance_predicted_3','electricity_pricing_predicted_1','electricity_pricing_predicted_2','electricity_pricing_predicted_3'
]

df_cleaned = F.drop(columns=columns_to_remove)

# Method 2: By column positions (alternative)
# df_cleaned = df.drop(df.columns[[4, 5, 6]], axis=1)

# Save the cleaned data back to Excel
df_cleaned.to_csv(r'C:\Users\PMLS\shahid (ML-lab)\dataset\test_data1.csv', index=False)

print("Columns E, F, G have been successfully removed and saved to cleaned_citylearn_data.xlsx")

Columns E, F, G have been successfully removed and saved to cleaned_citylearn_data.xlsx


In [3]:
dt = pd.read_csv('cleaned_citylearn_data_all_outliers_handled.csv')
dt.shape

(8760, 26)

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Load dataset
dt = pd.read_csv('cleaned_citylearn_data_all_outliers_handled.csv')

# Extract datetime features
if 'datetime' in dt.columns:
    dt['datetime'] = pd.to_datetime(dt['datetime'])
    dt['month'] = dt['datetime'].dt.month
    dt['hour'] = dt['datetime'].dt.hour
    dt['day_type'] = dt['datetime'].dt.dayofweek  # Monday=0, Sunday=6
else:
    print("No datetime column found; please ensure temporal features exist")

# Split dataset into train, validation, test using your function sp.t_v_t()
train_set, validation_set, test_set = sp.t_v_t(dt, 90, 5, 5)

# ---- Cyclical Encoding Function ----
def cyclical_encode(df, column, max_value):
    df[f"{column}_sin"] = np.sin(2 * np.pi * df[column] / max_value)
    df[f"{column}_cos"] = np.cos(2 * np.pi * df[column] / max_value)
    return df

# Apply cyclical encoding to all sets for same columns
for dataset in [train_set, validation_set, test_set]:
    for col, max_val in [('month', 12), ('hour', 24), ('day_type', 7)]:
        if col in dataset.columns:
            cyclical_encode(dataset, col, max_val)
    # Drop original temporal columns after encoding
    dataset.drop(columns=['month', 'hour', 'day_type'], errors='ignore', inplace=True)

# Define feature groups
#weather_features = [
#    'outdoor_dry_bulb_temperature', 'outdoor_dry_bulb_temperature_predicted_1','outdoor_dry_bulb_temperature_predicted_2','outdoor_dry_bulb_temperature_predicted_3',
 #   'outdoor_relative_humidity', 'outdoor_relative_humidity_predicted_1','outdoor_relative_humidity_predicted_2','outdoor_relative_humidity_predicted_3',
  #  'diffuse_solar_irradiance', 'diffuse_solar_irradiance_predicted_1','diffuse_solar_irradiance_predicted_2','diffuse_solar_irradiance_predicted_3',
   # 'direct_solar_irradiance', 'direct_solar_irradiance_predicted_1','direct_solar_irradiance_predicted_2','direct_solar_irradiance_predicted_3']

econ_env_features = [
    'non_shiftable_load',
    'electricity_pricing', 'electricity_pricing_predicted_1','electricity_pricing_predicted_2','electricity_pricing_predicted_3',
    'solar_generation',
    'carbon_intensity', 'outdoor_dry_bulb_temperature', 'outdoor_dry_bulb_temperature_predicted_1','outdoor_dry_bulb_temperature_predicted_2','outdoor_dry_bulb_temperature_predicted_3',
    'outdoor_relative_humidity', 'outdoor_relative_humidity_predicted_1','outdoor_relative_humidity_predicted_2','outdoor_relative_humidity_predicted_3',
    'diffuse_solar_irradiance', 'diffuse_solar_irradiance_predicted_1','diffuse_solar_irradiance_predicted_2','diffuse_solar_irradiance_predicted_3',
    'direct_solar_irradiance', 'direct_solar_irradiance_predicted_1','direct_solar_irradiance_predicted_2','direct_solar_irradiance_predicted_3'
]

# Filter features to those that actually exist in train set (to avoid errors)
#weather_features = [f for f in weather_features if f in train_set.columns]
econ_env_features = [f for f in econ_env_features if f in train_set.columns]

# ---- Scaling ----

# 2) Economic/Environmental features - MinMaxScaler (fit on train, transform on all)
econ_env_scaler = MinMaxScaler()
train_set[econ_env_features] = econ_env_scaler.fit_transform(train_set[econ_env_features])
validation_set[econ_env_features] = econ_env_scaler.transform(validation_set[econ_env_features])
test_set[econ_env_features] = econ_env_scaler.transform(test_set[econ_env_features])

# Save normalized datasets
train_set.to_csv(r'C:\Users\PMLS\shahid (ML-lab)\final_data\train_data.csv', index=False)
validation_set.to_csv(r'C:\Users\PMLS\shahid (ML-lab)\final_data\validation_data.csv', index=False)
test_set.to_csv(r'C:\Users\PMLS\shahid (ML-lab)\final_data\test_data.csv', index=False)

print("Normalized train, validation, and test datasets saved successfully.")


No datetime column found; please ensure temporal features exist


C:\Users\PMLS\anaconda3\envs\mll\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\PMLS\anaconda3\envs\mll\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\PMLS\anaconda3\envs\mll\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

Normalized train, validation, and test datasets saved successfully.


In [8]:
tr = pd.read_csv(r"C:\Users\PMLS\shahid (ML-lab)\final_data\train_data.csv")
vl = pd.read_csv(r"C:\Users\PMLS\shahid (ML-lab)\final_data\validation_data.csv")
tst = pd.read_csv(r"C:\Users\PMLS\shahid (ML-lab)\final_data\test_data.csv")

In [9]:
# Load your Excel file

#df = pd.read_excel('merged_citylearn_data.xlsx')
# Load dataset
#dt = pd.read_csv('train_data.csv')
#optional_features = ['electricity_pricing_predicted_2','outdoor_relative_humidity_predicted_1','outdoor_dry_bulb_temperature_predicted_1','day_type','month','outdoor_relative_humidity_predicted_3']

# Drop optional features
#df_optional_removed = df.drop(columns=optional_features)

# Method 1: By column names (recommended)
low_correlation_features = [
    'outdoor_relative_humidity_predicted_2',
    'outdoor_relative_humidity',
    'outdoor_dry_bulb_temperature',
    'outdoor_dry_bulb_temperature_predicted_2',
    'outdoor_dry_bulb_temperature_predicted_3',
    'diffuse_solar_irradiance_predicted_2',
    'direct_solar_irradiance_predicted_2',
    'electricity_pricing_predicted_1',
    'diffuse_solar_irradiance_predicted_3',
    'direct_solar_irradiance_predicted_3',
    'diffuse_solar_irradiance',
    'direct_solar_irradiance',
    'solar_generation'
]

# Drop low correlation features
df_low_removed1 = tr.drop(columns=low_correlation_features)
df_low_removed2 = vl.drop(columns=low_correlation_features)
df_low_removed3= tst.drop(columns=low_correlation_features)


# Method 2: By column positions (alternative)
# df_cleaned = df.drop(df.columns[[4, 5, 6]], axis=1)

# Save the cleaned data back to Excel
df_low_removed1.to_csv(r'C:\Users\PMLS\shahid (ML-lab)\final_data\train_data1.csv', index=False)
df_low_removed2.to_csv(r'C:\Users\PMLS\shahid (ML-lab)\final_data\validation_data1.csv', index=False)
df_low_removed3.to_csv(r'C:\Users\PMLS\shahid (ML-lab)\final_data\test_data1.csv', index=False)
print("Columns E, F, G have been successfully removed and saved to cleaned_citylearn_data.xlsx")

Columns E, F, G have been successfully removed and saved to cleaned_citylearn_data.xlsx


In [10]:
df_low_removed3.shape

(438, 16)